In [1]:
import urllib.request
from bs4 import BeautifulSoup as soup
import requests
import re
import pandas as pd
import dataframe_image as dfi


link = 'https://rally-base.com/2023/rally-estonia-2023/?ssId='
startat, no_ss=7729, int(21) # starting number of url, count of Special Stages
canceled = []

stages = [number for number in range(no_ss)]


rally_23 = []

In [2]:
if canceled:
    for j in canceled: stages.remove(j-1)
#print(stages)


In [3]:
for ss in stages:
    val= startat + ss
    ss_a = str(val)
    my_url11 = link + ss_a
    #print(ss, val, ss_a, "\n", my_url11)
    
    req = urllib.request.Request(my_url11, headers={'User-Agent': 'Mozilla/5.0'})
    uClient11 = urllib.request.urlopen(req)
    page_html11 = uClient11.read()
    uClient11.close()
    data = pd.read_html(page_html11)[1]
   
    data.columns = data.iloc[0]
    data = data[1:]
    data['ss']=ss+1
    #print(data.columns)
    
    equal = '-' in data['Pos.'].unique()
    if equal:
        data['Pos.'] = data['Pos.'].replace('-', method='ffill')
    
    #data.to_csv('05_rally23_SS'+str(ss+1)+'.csv', index=False)

    rally_23.append(data)

In [4]:
rally2023_stages = pd.concat(rally_23, axis=0)
#rally2023_stages['Pos.'] = rally2023_stages['Pos.'].astype(str).astype(int)
rally2023_stages['No.'] = rally2023_stages['No.'].astype(str).astype(int)
#rally2023_stages.to_csv('03_rally2023.csv', index=False)
rally2023_stages = rally2023_stages.fillna("-")
rally2023_stages

,Pos.,No.,Driver / Co-driver Vehicle,Group,SS time,Aver. speed sec/km,Diff.Leader Diff.Prev.,ss
1,1,8,Tänak Ott / Järveoja M. Ford Puma Rally1 ...,RC1,00:03:02.5,66.08 0.00,00:00:00.0 00:00:00.0,1
2,2,4,Lappi Esapekka / Ferm Janne Hyundai i20 N...,RC1,00:03:03.1,65.87 0.18,00:00:00.6 00:00:00.6,1
3,2,33,Evans Elfyn / Martin Scott Toyota GR Yari...,RC1,00:03:03.1,65.87 0.18,00:00:00.6 00:00:00.0,1
4,4,69,Rovanperä K. / Halttunen Jonne Toyota GR ...,RC1,00:03:03.2,65.83 0.21,00:00:00.7 00:00:00.1,1
5,5,11,Neuville T. / Wydaeghe M. Hyundai i20 N R...,RC1,00:03:04.1,65.51 0.48,00:00:01.6 00:00:00.9,1
...,...,...,...,...,...,...,...,...
40,40,59,Anwar Hamza / Kihurani A. Ford Fiesta Rally3,RC3,00:11:34.6,95.88 7.55,00:02:19.7 00:00:20.4,21
41,41,55,Blach Roberto / Barreiro Mauro Ford Fiest...,RC3,00:11:38.9,95.29 7.78,00:02:24.0 00:00:04.3,21
42,42,51,Cumiskey B. / Kierans Arthur Ford Fiesta ...,RC3,00:11:54.9,93.16 8.65,00:02:40.0 00:00:16.0,21
43,43,45,Cobbe Luciano / Mometti Roberto Škoda Fab...,RC2,00:12:08.0,91.48 9.35,00:02:53.1 00:00:13.1,21


In [5]:
rally2023_stageClass=rally2023_stages.drop(['Aver. speed sec/km', 'Diff.Leader Diff.Prev.', 'SS time'], axis=1)
rally2023_stageTimes=rally2023_stages.drop(['Aver. speed sec/km', 'Diff.Leader Diff.Prev.', 'Pos.'], axis=1)
rally2023_ClassView = rally2023_stageClass.set_index(['No.','Driver / Co-driver Vehicle','Group','ss'], drop=True).unstack('ss')
rally2023_TimeView = rally2023_stageTimes.set_index(['No.','Driver / Co-driver Vehicle','Group', 'ss'], drop=True).unstack('ss')
rally2023_ClassView = rally2023_ClassView.fillna("-")
rally2023_TimeView = rally2023_TimeView.fillna("-")
rally2023_stageClass2 = rally2023_stageClass.loc[((rally2023_stageClass['Group'] == 'RC1') | (rally2023_stageClass['Group'] == 'RC2'))] 
rally2023_ClassView2 = rally2023_stageClass2.set_index(['No.','Driver / Co-driver Vehicle','Group','ss'], drop=True).unstack('ss')
rally2023_ClassView2 = rally2023_ClassView2.fillna("-")

In [6]:
rally2023_ClassView.to_csv('08_Estonia_Stages_Classification.csv')
rally2023_TimeView.to_csv('08_Estonia_Stages_Times.csv',index=False)

In [7]:
pd.set_option("display.max_columns", None)

dfi.export(rally2023_ClassView2, 'Estonia_Stages_Classification.jpg')

In [10]:
rally2023_TimeView

0                                                                SS time  \
ss                                                                    1    
No. Driver / Co-driver Vehicle                         Group               
3   Suninen  Teemu  /  Markkula  Mikko  Hyundai i20... RC1    00:03:07.0   
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ... RC1    00:03:03.1   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall... RC1    00:03:07.5   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H... RC1    00:03:02.5   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra... RC1    00:03:04.1   
18  Katsuta  T.  /  Johnston  Aaron  Toyota GR Yari... RC1    00:03:05.7   
20  Solberg  Oliver  /  Edmondson  E.  Škoda Fabia ... RC2    00:03:10.0   
21  Greensmith  Gus  /  Andersson  Jonas  Škoda Fab... RC2    00:03:11.4   
22  Lindholm  Emil  /  Hämäläinen  R.  Hyundai i20 ... RC2    00:03:15.7   
23  Mikkelsen  A.  /  Eriksen  T.  Škoda Fabia RS R... RC2    00:03:09.7   
24  Pajari  Sami  /  Mälkönen  Enni  Škoda Fabia RS... RC2    00:03:10.4   
25  Kaur  Egon  /  Viilo  Jakko  Škoda Fabia RS Rally2 RC2    00:03:13.3   
26  Virves  Robert  /  Drew  Craig  Ford Fiesta Ral... RC2    00:03:11.6   
27  Bulacia  Marco  /  Vallejo  Diego  Škoda Fabia ... RC2    00:03:13.1   
28  Linnamäe  Georg  /  Morgan  James  Hyundai i20 ... RC2    00:03:09.1   
29  Marczyk  Miko  /  Gospodarczyk  S.  Škoda Fabia... RC2    00:03:09.0   
30  Zaldivar  F.  /  Der Ohannesian  M.  Hyundai i2... RC2    00:03:12.8   
31  McErlean  Joshua  /  Fulton  James  Hyundai i20... RC2    00:03:11.0   
33  Evans  Elfyn  /  Martin  Scott  Toyota GR Yaris... RC1    00:03:03.1   
34  Martínez  Jorge  /  Álvarez  A.  Škoda Fabia Ra... RC2    00:03:13.9   
35  Bulacia  Bruno  /  Coronado  Axel  Škoda Fabia ... RC2    00:03:11.6   
36  Fernàndez  E.  /  Rozada  Borja  Škoda Fabia Ra... RC2    00:03:11.9   
37  Koik  Priit  /  Tamm  Kristo  Škoda Fabia Rally... RC2    00:03:18.9   
38  Kasari  Kaspar  /  Raidma  Rainis  Škoda Fabia R5  RC2    00:03:17.5   
39  Villanueva  A.  /  Murado  José  Škoda Fabia RS... RC2    00:03:21.3   
40  Kremer  Armin  /  Kremer  Ella  Škoda Fabia RS ... RC2    00:13:09.0   
41  Miele  Mauro  /  Beltrame  Luca  Škoda Fabia RS... RC2    00:03:18.8   
45  Cobbe  Luciano  /  Mometti  Roberto  Škoda Fabi... RC2    00:03:27.4   
47  Korhonen  Roope  /  Viinikka  Anssi  Ford Fiest... RC3    00:03:17.1   
48  Herranen  Toni  /  Lukka  Mikko  Ford Fiesta Ra... RC3    00:03:19.2   
49  Türkkan  Ali  /  Erdener  Burak  Ford Fiesta Ra... RC3    00:03:19.3   
50  Korhola  B.  /  Kelander  Pekka  Ford Fiesta Ra... RC3    00:03:13.9   
51  Cumiskey  B.  /  Kierans  Arthur  Ford Fiesta R... RC3    00:03:25.7   
52  Creighton  W.  /  Regan  Liam  Ford Fiesta Rally3  RC3    00:03:14.8   
53  Dominguez  Diego  /  Peñate  Rogelio  Ford Fies... RC3    00:03:14.3   
54  Pellier  Laurent  /  Bronner  Kévin  Ford Fiest... RC3    00:03:15.4   
55  Blach  Roberto  /  Barreiro  Mauro  Ford Fiesta... RC3    00:03:25.1   
56  Rensonnet  Tom  /  Dumont  Loïc  Ford Fiesta Ra... RC3    00:03:24.2   
57  Kelly  Eamonn  /  Mohan  Conor  Ford Fiesta Rally3 RC3    00:03:21.0   
58  Munster  G.  /  Louka  Louis  Ford Fiesta Rally3   RC3    00:03:16.8   
59  Anwar  Hamza  /  Kihurani  A.  Ford Fiesta Rally3  RC3    00:03:23.3   
60  Mauro  Alejandro  /  Sanjuán  Diego  Škoda Fabi... RC2    00:03:18.7   
62  Kohn  Filip  /  Woodburn  Tom  Ford Fiesta Rally3  RC3    00:03:23.6   
63  Vaher  Jaspar  /  Mitendorf  E.  Ford Fiesta Ra... RC4    00:03:30.6   
64  Sei  Karl-Markus  /  Leotoots  Martin  Peugeot ... RC4    00:03:27.3   
65  Jürgenson  R.  /  Oja  Siim  Ford Fiesta Rally4    RC4    00:03:31.0   
66  Lempu  Kevin  /  Rahumeel  Andre  Ford Fiesta R... RC4    00:03:35.4   
67  Uger  Sergey  /  Uger  Maria  Ford Fiesta R5       RC2    00:03:39.4   
69  Rovanperä  K.  /  Halttunen  Jonne  Toyota GR Y... RC1    00:03:03.2   

0                                              